In [3]:
pip install tensorflow==2.4.0


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.0 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0)
ERROR: No matching distribution found for tensorflow==2.4.0


In [5]:
pip install tensorflow-quantum

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow-quantum (from versions: none)
ERROR: No matching distribution found for tensorflow-quantum


In [6]:
pip install cirq

  Using cached cirq-1.1.0-py3-none-any.whl (7.7 kB)
  Using cached cirq_aqt-1.1.0-py3-none-any.whl (27 kB)
  Using cached cirq_web-1.1.0-py3-none-any.whl (594 kB)
  Using cached cirq_ionq-1.1.0-py3-none-any.whl (57 kB)
  Using cached cirq_pasqal-1.1.0-py3-none-any.whl (31 kB)
  Using cached cirq_rigetti-1.1.0-py3-none-any.whl (66 kB)
  Using cached cirq_google-1.1.0-py3-none-any.whl (577 kB)
  Using cached cirq_core-1.1.0-py3-none-any.whl (1.8 MB)
  Using cached duet-0.2.7-py3-none-any.whl (28 kB)
  Using cached proto_plus-1.22.2-py3-none-any.whl (47 kB)
  Using cached google_api_core-1.34.0-py3-none-any.whl (120 kB)
  Using cached protobuf-3.20.3-cp39-cp39-win_amd64.whl (904 kB)
  Using cached grpcio_status-1.51.3-py3-none-any.whl (5.1 kB)
  Using cached grpcio_status-1.51.1-py3-none-any.whl (5.1 kB)
  Using cached grpcio_status-1.50.0-py3-none-any.whl (14 kB)
  Using cached grpcio_status-1.49.1-py3-none-any.whl (14 kB)
  Using cached grpcio_status-1.48.2-py3-none-any.whl (14 kB)
  Us

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\anyio-2.2.0.dist-info\\direct_url.json'
Consider using the `--user` option or check the permissions.



In [10]:
import pennylane as qml
from pennylane import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split

In [11]:
# Define the graph and its adjacency matrix
G = nx.Graph()
G.add_edges_from([(0, 1), (0, 2), (1, 2), (2, 3)])
adj_matrix = nx.adjacency_matrix(G).todense()

In [12]:
# Define the QGNN circuit
dev = qml.device('default.qubit', wires=4)
num_layers = 2


In [13]:
@qml.qnode(dev)
def qgnn_circuit(inputs, weights):
    for i in range(num_layers):
        for j in range(4):
            qml.RY(inputs[j], wires=j)
        for j in range(4):
            for k in range(j+1, 4):
                if adj_matrix[j, k] == 1:
                    qml.CNOT(wires=[j, k])
            qml.RZ(weights[i][j], wires=j)
    return [qml.expval(qml.PauliZ(j)) for j in range(4)]


In [14]:
# Define the cost function
def cost(weights, inputs, labels):
    predictions = [qgnn_circuit(inputs[i], weights) for i in range(inputs.shape[0])]
    return np.sum([(predictions[i][0] - labels[i])**2 for i in range(inputs.shape[0])])


In [16]:
# Load the ParticleNet dataset
data = np.load('QG_jets.npz')
features = data['X']
labels = data['y']
inputs_train, inputs_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2)

# Initialize the weights
weights = np.random.randn(num_layers, 4)

# Train the QGNN
opt = qml.GradientDescentOptimizer(stepsize=0.1)
epochs = 100
losses = []
for i in range(epochs):
    weights = opt.step(lambda w: cost(w, inputs_train, labels_train), weights)
    loss = cost(weights, inputs_train, labels_train)
    losses.append(loss)
    predictions = [qgnn_circuit(inputs_train[i], weights) for i in range(inputs_train.shape[0])]
    accuracy = np.sum([np.round(predictions[i][0]) == labels_train[i] for i in range(inputs_train.shape[0])])/inputs_train.shape[0]
    print(f"Epoch {i+1}/{epochs}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}")

# Test the QGNN
predictions = [qgnn_circuit(inputs_test[i], weights) for i in range(inputs_test.shape[0])]
accuracy = np.sum([np.round(predictions[i][0]) == labels_test[i] for i in range(inputs_test.shape[0])])/inputs_test.shape[0]
print("Test Accuracy:", accuracy)

Epoch 1/100: Loss = 115885.7201, Accuracy = 2.0178
Epoch 2/100: Loss = 114986.7586, Accuracy = 2.0276
Epoch 3/100: Loss = 188803.9091, Accuracy = 1.7915
Epoch 4/100: Loss = 153256.3573, Accuracy = 1.9701
Epoch 5/100: Loss = 125853.4552, Accuracy = 2.0072
Epoch 6/100: Loss = 118542.0017, Accuracy = 2.0270
Epoch 7/100: Loss = 120112.2204, Accuracy = 2.0112
Epoch 8/100: Loss = 115436.8218, Accuracy = 2.0285


KeyboardInterrupt: 